Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

In [ ]:
import csv

with open('checkins.dat') as dat_file, open('checkins.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас.

In [13]:
import pandas as pd
from pandas import DataFrame
data = pd.read_csv('checkins.csv')

coordinates = DataFrame(data, columns= ['latitude','longitude'])

coordinates_sample = coordinates[:100000]
coordinates_sample.head()

,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [17]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth

ms = MeanShift(bandwidth=0.1)
ms.fit(coordinates_sample)

cluster_map = pd.DataFrame()
cluster_map['data_index'] = coordinates_sample.index.values
cluster_map['cluster'] = ms.labels_

labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)

number of estimated clusters : 3230


Определим центры кластеров, чей размер более 15ти.

In [25]:
count_clusters = cluster_map.groupby(['cluster']).data_index.size()

count_clusters_15 = count_clusters[count_clusters > 15].index.tolist()

cluster_centers_15 = cluster_centers[count_clusters_15]
print(cluster_centers_15)

# print(len(cluster_centers_15))

[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ...
 [  41.61853175  -88.44556818]
 [  38.65877915  -76.8856871 ]
 [  39.2494686   -77.1821271 ]]


Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [34]:
offices =np.array([[33.751277, -118.188740],[25.867736, -80.324116], [51.503016, -0.075479],[52.378894, 4.885084],[39.366487, 117.036146],[-33.868457, 151.205134]]);
dist=[];
for i in range(cluster_centers_15.shape[0]):
    dist.append(np.min(np.linalg.norm(offices-cluster_centers_15[i,:],axis=1)))
np.min(dist),np.argmin(dist),cluster_centers_15[np.argmin(dist)]

min_distance = str(cluster_centers_15[np.argmin(dist)])[1:-1]

def write_answer_distance(a):
    with open("ans.txt","w") as file:
        file.write(a)
        
write_answer_distance(min_distance)